In [1]:
!nvidia-smi

Wed Jul 15 03:13:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 66%   67C    P0    75W / 250W |      0MiB / 11019MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 62%   66C    P0    79W / 250W |      0MiB / 11019MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
torch.Tensor(8)

tensor([1.5263e-36, 0.0000e+00, 1.5263e-36, 0.0000e+00, 1.5263e-36, 0.0000e+00,
        1.1351e-43, 0.0000e+00])

In [5]:
from torch.nn.parallel.data_parallel import DataParallel
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import os

os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29500'
dist.init_process_group(backend='nccl',rank=0,world_size=1)

In [6]:
from darts import Darts

net = Darts(C=36,num_classes=10,num_layers=8)



In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# CUDA 기기가 존재한다면, 아래 코드가 CUDA 장치를 출력합니다:

print(device)

cuda:0


In [8]:
net.to(device)
if torch.cuda.device_count() > 1:
    net = DDP(net,device_ids = [0,1,2,3])

In [9]:
# [x if 'arch' in x[0] else  for x in net.named_parameters()]
arch_params = []
weight_params = []
for x in net.named_parameters():
    if 'arch' in x[0]:
        arch_params.append(x[1])
    else:
        weight_params.append(x[1])


In [10]:
import torch.optim as optim
import torch
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer_weight = optim.SGD(weight_params, lr=0.01, momentum=0.9)
optimizer_arch = optim.Adam(arch_params, lr=0.001)

In [11]:
OPS = [
            'none',
            'max_pool_3x3',
            'avg_pool_3x3',
            'skip_connect',
            'sep_conv_3x3',
            'sep_conv_5x5',
            'dil_conv_3x3',
            'dil_conv_5x5'
        ]

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

num_classes = 10
trn_loss_list = []
val_loss_list = []
acc_list = []
acc_list_aug = []
num_batches=len(trainloader)
total_epoch = 50

for epoch in range(total_epoch):   # 데이터셋을 수차례 반복합니다.
    net.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0),total=len(trainloader)):
        if i%6 == 0 and epoch != 0:
            # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
            inputs, labels = data[0].to(device), data[1].to(device)

            # 변화도(Gradient) 매개변수를 0으로 만들고
            optimizer_arch.zero_grad()

            # 순전파 + 역전파 + 최적화를 한 후
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_arch.step()
            running_loss += loss.item()
        else:
            # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
            inputs, labels = data[0].to(device), data[1].to(device)

            # 변화도(Gradient) 매개변수를 0으로 만들고
            optimizer_weight.zero_grad()

            # 순전파 + 역전파 + 최적화를 한 후
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_weight.step()
            running_loss += loss.item()
#         print(loss.item())
        
    net.eval()
    with torch.no_grad(): # very very very very important!!!
        val_loss = 0.0
        class_correct = list(0. for i in range(num_classes))
        class_total = list(0. for i in range(num_classes))
        correct = 0
        total = 0
        for j,val in enumerate(testloader):
            v1 = val[0].to(device)
            val_labels = val[1].to(device)
            val_output = net(v1)
            v_loss = criterion(val_output, val_labels)
            val_loss += v_loss
            _, predicted = torch.max(val_output, 1)
            c = (predicted == val_labels).squeeze()
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
            for i in range(len(val_labels)):
                val_label = val_labels[i]
                class_correct[val_label] += c[i].item()
                class_total[val_label] += 1

        print("epoch:",str(epoch))
        temp_acc = []
        for i in range(num_classes):
            if class_total[i]==0:
                print('class_total = 0',class_correct,class_total)
            else:
                print('Accuracy of %5s : %2d %%' % (i, 100 * class_correct[i] / class_total[i]))
                temp_acc.append(100 * class_correct[i] / class_total[i])
        acc_list.append(temp_acc)
    
    print('arch_param_normal')
    for i in range(4):
        for op in torch.max(F.softmax(net.module.arch_param_normal[i],dim=1),1)[1]:
            print(OPS[op],end=' ')
        print()
#         print(torch.max(F.softmax(net.module.arch_param_normal[i],dim=1),1)[1])

    print()
    print('arch_param_reduc')
    for i in range(4):
        for op in torch.max(F.softmax(net.module.arch_param_reduc[i],dim=1),1)[1]:
            print(OPS[op],end=' ')
        print()

    print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
        epoch+1, total_epoch, i+1, num_batches, running_loss / len(trainloader), val_loss / len(testloader)
    ))        
    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

    trn_loss_list.append(running_loss/1875)
    val_loss_list.append(val_loss/len(testloader))
    running_loss = 0.0

print('Finished Training')

100%|██████████| 98/98 [02:52<00:00,  1.76s/it]


epoch: 0
Accuracy of     0 : 50 %
Accuracy of     1 : 37 %
Accuracy of     2 : 23 %
Accuracy of     3 : 13 %
Accuracy of     4 : 36 %
Accuracy of     5 : 17 %
Accuracy of     6 : 49 %
Accuracy of     7 : 73 %
Accuracy of     8 : 66 %
Accuracy of     9 : 67 %
arch_param_normal
sep_conv_3x3 none 
none none none 
max_pool_3x3 max_pool_3x3 max_pool_3x3 max_pool_3x3 
none none none none none 

arch_param_reduc
sep_conv_3x3 none 
none none none 
none none none none 
none none none none none 
epoch: 1/50 | step: 4/98 | trn loss: 1.6171 | val loss: 1.4980
Accuracy of the network on the test images: 43 %


100%|██████████| 98/98 [03:02<00:00,  1.86s/it]


epoch: 1
Accuracy of     0 : 70 %
Accuracy of     1 : 63 %
Accuracy of     2 : 28 %
Accuracy of     3 : 33 %
Accuracy of     4 : 56 %
Accuracy of     5 : 53 %
Accuracy of     6 : 37 %
Accuracy of     7 : 75 %
Accuracy of     8 : 47 %
Accuracy of     9 : 76 %
arch_param_normal
sep_conv_3x3 sep_conv_5x5 
max_pool_3x3 dil_conv_3x3 skip_connect 
max_pool_3x3 sep_conv_5x5 max_pool_3x3 max_pool_3x3 
sep_conv_5x5 dil_conv_3x3 max_pool_3x3 sep_conv_3x3 dil_conv_5x5 

arch_param_reduc
max_pool_3x3 none 
none sep_conv_3x3 dil_conv_5x5 
sep_conv_3x3 none sep_conv_3x3 sep_conv_5x5 
dil_conv_5x5 sep_conv_3x3 dil_conv_5x5 sep_conv_3x3 dil_conv_3x3 
epoch: 2/50 | step: 4/98 | trn loss: 1.2495 | val loss: 1.2590
Accuracy of the network on the test images: 54 %


100%|██████████| 98/98 [03:00<00:00,  1.84s/it]


...

100%|██████████| 98/98 [02:56<00:00,  1.80s/it]


epoch: 37
Accuracy of     0 : 69 %
Accuracy of     1 : 92 %
Accuracy of     2 : 69 %
Accuracy of     3 : 54 %
Accuracy of     4 : 78 %
Accuracy of     5 : 71 %
Accuracy of     6 : 81 %
Accuracy of     7 : 66 %
Accuracy of     8 : 92 %
Accuracy of     9 : 83 %
arch_param_normal
sep_conv_3x3 sep_conv_5x5 
sep_conv_5x5 sep_conv_5x5 sep_conv_5x5 
sep_conv_5x5 sep_conv_5x5 sep_conv_5x5 sep_conv_5x5 
sep_conv_5x5 max_pool_3x3 max_pool_3x3 sep_conv_5x5 max_pool_3x3 

arch_param_reduc
sep_conv_5x5 sep_conv_5x5 
sep_conv_5x5 sep_conv_3x3 dil_conv_5x5 
none none sep_conv_3x3 sep_conv_5x5 
none sep_conv_5x5 sep_conv_3x3 sep_conv_3x3 sep_conv_5x5 
epoch: 38/50 | step: 4/98 | trn loss: 0.0768 | val loss: 1.1331
Accuracy of the network on the test images: 75 %


100%|██████████| 98/98 [02:59<00:00,  1.84s/it]


epoch: 38
Accuracy of     0 : 71 %
Accuracy of     1 : 92 %
Accuracy of     2 : 52 %
Accuracy of     3 : 76 %
Accuracy of     4 : 77 %
Accuracy of     5 : 59 %
Accuracy of     6 : 79 %
Accuracy of     7 : 78 %
Accuracy of     8 : 90 %
Accuracy of     9 : 79 %
arch_param_normal
sep_conv_3x3 sep_conv_5x5 
sep_conv_5x5 sep_conv_5x5 sep_conv_5x5 
sep_conv_5x5 sep_conv_5x5 sep_conv_5x5 sep_conv_5x5 
sep_conv_5x5 max_pool_3x3 max_pool_3x3 sep_conv_5x5 max_pool_3x3 

arch_param_reduc
sep_conv_5x5 sep_conv_5x5 
sep_conv_5x5 sep_conv_3x3 dil_conv_5x5 
none none sep_conv_3x3 sep_conv_5x5 
none sep_conv_5x5 sep_conv_3x3 sep_conv_3x3 sep_conv_5x5 
epoch: 39/50 | step: 4/98 | trn loss: 0.0716 | val loss: 1.1922
Accuracy of the network on the test images: 75 %


 38%|███▊      | 37/98 [01:08<01:42,  1.69s/it]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.title('val_acc')
plt.plot([np.mean(x) for x in acc_list])
plt.legend()
plt.show()
plt.title('val_loss')
plt.plot(val_loss_list)
plt.legend()
plt.show()

In [ ]:
import torch.nn.functional as F

print('arch_param_normal')
for i in range(4):
    print(torch.max(F.softmax(net.module.arch_param_normal[i],dim=1),1)[1])
    
print()
print('arch_param_reduc')
for i in range(4):
    print(torch.max(F.softmax(net.module.arch_param_reduc[i],dim=1),1)[1])

In [ ]:
net.module.arch_param_reduc